In [1]:
import os
# to use or not to use GPU
#os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

# packages for model graph visualization
#!pip install -q pydot
# install graphviz https://graphviz.gitlab.io/download/ 

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub
!pip install -q -U tensorflow-text
import tensorflow_text as text
!pip install -q tf-models-official
from official.nlp import optimization  # to create AdamW optimizer

import json
import re
import random
import numpy as np
import matplotlib.pyplot as plt

#save_path = "/content/drive/My Drive/Colab Notebooks/"

     |████████████████████████████████| 4.9 MB 5.2 MB/s 
     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 42.5 MB/s 
     |████████████████████████████████| 213 kB 50.3 MB/s 
     |████████████████████████████████| 90 kB 9.1 MB/s 
     |████████████████████████████████| 43 kB 2.1 MB/s 
     |████████████████████████████████| 47.7 MB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 48.4 MB/s 
     |████████████████████████████████| 352 kB 54.3 MB/s 
     |████████████████████████████████| 99 kB 10.2 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
save_path = "/content/drive/My Drive/Colab Notebooks/"

In [4]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# checking if we have access to a GPU

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [6]:
#@title Data Loading & Preprocessing


In [7]:
with open(save_path+'kafka_data_preprocessed_high_occurrence_50.json') as f:
    data = json.load(f)
    # shuffle data
    random.shuffle(data)
    # create description and assignee list
    desc_data = []
    assignee_data = []
    for item in data:
        desc_data.append(item['description'])
        assignee_data.append(item['assignee'])

In [8]:
print('sample assignees:', assignee_data[0:2])
print('sample descriptions:', desc_data[0:2])

sample assignees: ['Jason Gustafson', 'A. Sophie Blee-Goldman']
sample descriptions: ['For KIP-320, we changed the permissions of the OffsetsForLeaderEpoch to be topic-level so that consumers did not require Cluster permission. However, there is no way for a consumer to know whether the broker is new enough to support this permission scheme. The only way to be sure is to use the version of this API that was bumped in 2.3. For older versions, we should revert to the old behavior.', "To avoid triggering auto topic creation (if `auto.create.topic.enable=true` on the brokers), Kafka Streams uses consumer pattern subscription. For this case, the consumer requests all metadata from the brokers and does client side filtering. However, if users want to set ACL to restrict a Kafka Streams application, this may results in broker side ERROR logs that some metadata cannot be provided. The only way to avoid those broker side ERROR logs is to grant corresponding permissions. As of 2.3 release it's p

In [9]:
# transform description data to tensorflow dataset
descriptions = tf.data.Dataset.from_tensor_slices(desc_data)
for input in descriptions.take(2):
    print(input)

tf.Tensor(b'For KIP-320, we changed the permissions of the OffsetsForLeaderEpoch to be topic-level so that consumers did not require Cluster permission. However, there is no way for a consumer to know whether the broker is new enough to support this permission scheme. The only way to be sure is to use the version of this API that was bumped in 2.3. For older versions, we should revert to the old behavior.', shape=(), dtype=string)
tf.Tensor(b"To avoid triggering auto topic creation (if `auto.create.topic.enable=true` on the brokers), Kafka Streams uses consumer pattern subscription. For this case, the consumer requests all metadata from the brokers and does client side filtering. However, if users want to set ACL to restrict a Kafka Streams application, this may results in broker side ERROR logs that some metadata cannot be provided. The only way to avoid those broker side ERROR logs is to grant corresponding permissions. As of 2.3 release it's possible to disable auto topic creation c

In [10]:
# target vectorize assignee data
assignee_dict = {assignee: i for i, assignee in enumerate(list(set(assignee_data)))}
assignee_dict



{'A. Sophie Blee-Goldman': 26,
 'Apurva Mehta': 33,
 'Ashish Singh': 7,
 'Bill Bejeck': 6,
 'Boyang Chen': 30,
 'Bruno Cadonna': 23,
 'Chia-Ping Tsai': 4,
 'Chris Egerton': 29,
 'Colin McCabe': 34,
 'Damian Guy': 32,
 'David Jacot': 20,
 'Dong Lin': 9,
 'Dongjin Lee': 19,
 'Eno Thereska': 2,
 'Ewen Cheslack-Postava': 5,
 'Grant Henke': 25,
 'Guozhang Wang': 14,
 'Gwen Shapira': 0,
 'Harsha': 8,
 'Ismael Juma': 1,
 'Jason Gustafson': 28,
 'Jay Kreps': 31,
 'Jiangjie Qin': 18,
 'John Roesler': 10,
 'Jun Rao': 16,
 'Konstantine Karantasis': 24,
 'Luke Chen': 3,
 'Manikumar': 27,
 'Matthias J. Sax': 21,
 'Neha Narkhede': 22,
 'Rajini Sivaram': 12,
 'Randall Hauch': 17,
 'Swapnil Ghike': 11,
 'Tom Bentley': 13,
 'Vahid Hashemian': 35,
 'huxihx': 15}

In [11]:
# create target vector
assignee_vector = [assignee_dict[assignee] for assignee in assignee_data]
assignee_vector[:10]

[28, 26, 11, 17, 34, 1, 7, 16, 31, 22]

In [12]:
# assignee target vector to one hot vector
assignee_tensor = tf.data.Dataset.from_tensor_slices(assignee_vector)
assignee_one_hot = assignee_tensor.map(lambda x: tf.one_hot(x, len(assignee_dict)))
for assignee in assignee_one_hot.take(5):
    print(assignee)

tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.], shape=(36,), dtype=float32)


In [13]:
# zip descriptions and assignee for training
dataset = tf.data.Dataset.zip((descriptions, assignee_one_hot))
for input, target in dataset.take(2):
    print(input)
    print(target)

tf.Tensor(b'For KIP-320, we changed the permissions of the OffsetsForLeaderEpoch to be topic-level so that consumers did not require Cluster permission. However, there is no way for a consumer to know whether the broker is new enough to support this permission scheme. The only way to be sure is to use the version of this API that was bumped in 2.3. For older versions, we should revert to the old behavior.', shape=(), dtype=string)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.], shape=(36,), dtype=float32)
tf.Tensor(b"To avoid triggering auto topic creation (if `auto.create.topic.enable=true` on the brokers), Kafka Streams uses consumer pattern subscription. For this case, the consumer requests all metadata from the brokers and does client side filtering. However, if users want to set ACL to restrict a Kafka Streams application, this may results in broker side ERROR logs that some metadata cannot be provided. The

In [14]:
# shuffle dataset
data_len = len(dataset)
dataset = dataset.shuffle(data_len)


In [15]:
# split dataset into train val and test
BATCH_SIZE = 32
VAL_SIZE = 0.15
n_examples = tf.data.experimental.cardinality(dataset).numpy()
print('n_examples:', n_examples)

VAL_SIZE = round(VAL_SIZE*n_examples)
test_dataset = dataset.take(VAL_SIZE).batch(batch_size=BATCH_SIZE, drop_remainder=True)
val_dataset = dataset.skip(VAL_SIZE).take(VAL_SIZE).batch(batch_size=BATCH_SIZE, drop_remainder=True)
train_dataset = dataset.skip(VAL_SIZE*2).batch(batch_size=BATCH_SIZE, drop_remainder=True)


n_examples: 4338


In [16]:
# prefetch train val and test dataset
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

print('len(train_dataset):', len(train_dataset))
print('len(val_dataset):', len(val_dataset))
print('len(test_dataset):', len(test_dataset))


len(train_dataset): 94
len(val_dataset): 20
len(test_dataset): 20


In [17]:
# some train dataset examples
for input, target in train_dataset.take(2):
    print(input)
    print(target)

tf.Tensor(
[b"Users often want to read topic multiple times. However, this is not possible because there is a single consumer and thus a topic can only be consumed once. Users get an exception {quote}Exception in thread \xe2\x80\x9cmain\xe2\x80\x9d org.apache.kafka.streams.errors.TopologyException: Invalid topology: Topic source has already been registered by another source. {quote} If they use a topic name in multiple `stream()`, `table()`, `globalTable()` calls. However, with KAFKA-6034 in place, we could allow adding a topic multiple times and rewrite the topology internally to only read the topic once. This would simplify application code as users don't need to put workaround in place to get the same behavior."
 b'Currently it does not appear that the KafkaShortnamer allows for case insensitive search and replace rules. It would be good to match the functionality provided by HDFS as operators are familiar with this. This also makes it easier to port auth_to_local rules from your ex